In [9]:
import cv2
import mediapipe as mp
from matplotlib import pyplot as plt
import numpy as np
from scipy import sparse
import math
import torch
import import_ipynb
import import_ipynb
from localAngle import calc_localAngle
from coordinate import coordinateFeature
from coordinate import normFunction
from distanceNode import distance_node_feature

In [10]:
def Angle(BA,BC):
    modBA= math.sqrt(BA[0]**2+BA[1]**2+BA[2]**2)
    modBC= math.sqrt(BC[0]**2+BC[1]**2+BC[2]**2)
    BABC= modBA * modBC
    dotProduct=BA[0]*BC[0] + BA[1]*BC[1] + BA[2]*BC[2]
    x = dotProduct / BABC
    angle = math.acos(x)
    return(math.degrees(angle))

In [11]:
# Calculating Local Angle For Node Feature
def fetch_local_angles(BINARY_ADJACENCY_MATRIX, x_axis, y_axis, z_axis):
    row_indices, col_indices = BINARY_ADJACENCY_MATRIX.nonzero()
    Local_Angle_features=[]
    for landmark in range(468):
        List = []
        for i in range(len(row_indices)):
            if(row_indices[i] == landmark):
                List.append([x_axis[col_indices[i]], y_axis[col_indices[i]], z_axis[col_indices[i]]])
        A = [x_axis[landmark], y_axis[landmark], z_axis[landmark]]
        # print("A=",A, "S=",List)
        Local_Angle_features.append(calc_localAngle(A, np.array(List)))
    # print("Local_Angle_features=",Local_Angle_features)
    return normFunction(Local_Angle_features)

In [12]:
# Calculating distance For Node Feature
def fetch_distance_node_feature(BINARY_ADJACENCY_MATRIX, x_axis, y_axis, z_axis):
    row_indices, col_indices = BINARY_ADJACENCY_MATRIX.nonzero()
    # print( row_indices, col_indices)
    Distance_node_features=[]
    for landmark in range(468):
        List = []
        for i in range(len(row_indices)):
            if(row_indices[i] == landmark):
                List.append([x_axis[col_indices[i]], y_axis[col_indices[i]], z_axis[col_indices[i]]])
        A = [x_axis[landmark], y_axis[landmark], z_axis[landmark]]
        Distance_node_features.append(distance_node_feature(A, np.array(List)))
    # print("Local_Angle_features=",Local_Angle_features)
    return normFunction(Distance_node_features)

In [13]:
def mediapipe(path): 
    image=cv2.imread(path)
    image=cv2.resize(image,(600,600))
    annoted_image=image.astype(np.uint8)

    mp_face_mesh=mp.solutions.face_mesh
    connection_tesselation=mp_face_mesh.FACEMESH_TESSELATION
    # print("edges_mediapipe=",len(connection_tesselation),len(connection_tesselation)/2)
    # print(connection_tesselation)
    with mp_face_mesh.FaceMesh(static_image_mode=False,max_num_faces=2,refine_landmarks=True,min_detection_confidence=0.5) as face_mesh:
        results=face_mesh.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))

    x_axis=np.empty(len(results.multi_face_landmarks[0].landmark))
    y_axis=np.empty(len(results.multi_face_landmarks[0].landmark))
    z_axis=np.empty(len(results.multi_face_landmarks[0].landmark))
    i=0

    for nodes in results.multi_face_landmarks[0].landmark:
        x_axis[i]=(results.multi_face_landmarks[0].landmark[i].x)
        y_axis[i]=(results.multi_face_landmarks[0].landmark[i].y)
        z_axis[i]=(results.multi_face_landmarks[0].landmark[i].z)
        i=i+1
    # print(len(results.multi_face_landmarks[0].landmark))
    # print("xyz=",x_axis)

    FEATURE_MATRIX=np.zeros((468,468))
    for i in range(468):
        FEATURE_MATRIX[i][i]=1

    WEIGHTED_ADJACENCY_MATRIX=np.zeros((468,468)) #creating a numpy array of shape 468X468 initialized with zero
    BINARY_ADJACENCY_MATRIX=np.zeros((468,468))
    ANGLE_BETWEEN_EDGE_FEATURE=np.zeros((468,468))

    for edge in connection_tesselation:
        # Determining X & Y axis of Two Connected Points
        x1=x_axis[edge[0]]
        y1=y_axis[edge[0]]
        z1=z_axis[edge[0]]
        x2=x_axis[edge[1]]
        y2=y_axis[edge[1]]
        z2=z_axis[edge[1]]
        # Calculating Eucleadin Distance for Weighted Graph
        eucleadian_distance=(((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2) ** 0.5)
        # Creating Weighted Graph
        WEIGHTED_ADJACENCY_MATRIX[edge[0]][edge[1]]=eucleadian_distance
        WEIGHTED_ADJACENCY_MATRIX[edge[1]][edge[0]]=eucleadian_distance

        BINARY_ADJACENCY_MATRIX[edge[0]][edge[1]]=1
        BINARY_ADJACENCY_MATRIX[edge[1]][edge[0]]=1

        angle_betn_edge=Angle([x1, y1, z1],[x2, y2, z2])
        ANGLE_BETWEEN_EDGE_FEATURE[edge[0]][edge[1]]=angle_betn_edge
        ANGLE_BETWEEN_EDGE_FEATURE[edge[1]][edge[0]]=angle_betn_edge

        # Normalizing axis Values For Visualizing Meash on Face
        shape = image.shape 
        relative_x = int(x1 * shape[1])
        relative_y = int(y1 * shape[0])
        relative_z = int(x2 * shape[1])
        relative_c = int(y2 * shape[0])
        # Drawing Nodes and Edges on Image For Visualization
        cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 0, 255), thickness=3) 
        cv2.line(image,(relative_x,relative_y),(relative_z,relative_c),(0,255,0),1)

    # plt.imshow(image)
    WEIGHTED_ADJACENCY_MATRIX=sparse.csr_matrix(WEIGHTED_ADJACENCY_MATRIX)
    BINARY_ADJACENCY_MATRIX=sparse.csr_matrix(BINARY_ADJACENCY_MATRIX)
    # print(BINARY_ADJACENCY_MATRIX)
    LocalAngles = fetch_local_angles(BINARY_ADJACENCY_MATRIX, x_axis, y_axis, z_axis)
    Distance_node_feature = fetch_distance_node_feature(BINARY_ADJACENCY_MATRIX, x_axis, y_axis, z_axis)
    # print(Distance_node_feature)
    XCoordinateFeature, YCoordinateFeature, ZCoordinateFeature = coordinateFeature(x_axis, y_axis, z_axis)
    # FEATURE_MATRIX=torch.from_numpy(FEATURE_MATRIX)
    DOTPRODUCT=WEIGHTED_ADJACENCY_MATRIX.dot(BINARY_ADJACENCY_MATRIX)
    # newFeatureMatrix = LocalAngles
    newFeatureMatrix = torch.from_numpy(np.column_stack((LocalAngles,  XCoordinateFeature[:-10], YCoordinateFeature[:-10], ZCoordinateFeature[:-10], Distance_node_feature, FEATURE_MATRIX)))
    # print( LocalAngles[0], XCoordinateFeature[:-10][0], YCoordinateFeature[:-10][0], ZCoordinateFeature[:-10][0])
    return(WEIGHTED_ADJACENCY_MATRIX,newFeatureMatrix)
    # return(DOTPRODUCT,newFeatureMatrix/np.linalg.norm(newFeatureMatrix))


Checking mediapipe is running currectly or not

In [14]:
# path="./datasets/orl_dataset/person10/test_image/98_10.jpg"
# adj1,features1=mediapipe(path)
# for i in range(50):
#     print(features1[i])



[-0.658498085574667, -0.7274305630648765, -0.5177836568012124, -0.5175567522323883, -0.4796079050587039, -0.5596011780547775, -0.4183970016868238, -0.9061566227536618, -0.11353445513434979, 0.4930154980354673, 0.7165463597714476, -0.6250295490065725, -0.6148045672561073, -0.5747524022481217, -0.632269264331984, -0.6569710702252329, -0.7171244895040734, -0.34508432054273497, -0.3253765716807808, -0.5779809523339692, -0.5628470891130879, 0.7418641891086724, -0.6770901786334599, -0.6135481425852023, -0.5900635460740296, -0.7194967201155127, -0.8320015834695464, -0.5894895396523101, -0.5169940923819601, -0.67281368562317, -0.7018756535811274, -0.547775785319276, -0.21133955181279007, -0.8872465329855903, 0.8156131216097733, -0.2800683559717264, -0.17394933761349818, -0.5706066135900107, -0.625177901530299, -0.48273598769102555, -0.5349306928365078, -0.6843616973132116, -0.7551511584878057, -0.44379727536036206, -0.6168259513320768, -0.5022226666038647, -0.3103282343292978, -0.5414241757434

In [15]:
# adj1=adj1.toarray()
# adj2=adj2.toarray()

In [16]:
# path="./datasets/orl_dataset/person9/test_image/88_9.jpg"
# adj1,features1=mediapipe(path)
# for i in range(50):
#     print(features1[i])
